In [ ]:
import pandas as pd
ipl_auction_df = pd.read_csv("IPL_IMB381IPL2013.csv")
#print("ipl_auction_df")
#print(ipl_auction_df.head())
#print("ipl_auction_df.shape")
#print(ipl_auction_df.shape)

# List columns in the DataFrame
print("ipl_auction_df.columns")
print(ipl_auction_df.columns)


ipl_auction_df.iloc[0:5, 0:10]  

X_features = ipl_auction_df.columns


ipl_auction_df
   Sl.NO.   PLAYER NAME  AGE COUNTRY  TEAM PLAYING ROLE  T-RUNS  T-WKTS  \
0       1   Abdulla, YA    2      SA  KXIP   Allrounder       0       0   
1       2  Abdur Razzak    2     BAN   RCB       Bowler     214      18   
2       3   Agarkar, AB    2     IND   KKR       Bowler     571      58   
3       4     Ashwin, R    1     IND   CSK       Bowler     284      31   
4       5  Badrinath, S    2     IND   CSK      Batsman      63       0   

   ODI-RUNS-S  ODI-SR-B  ...    SR-B  SIXERS  RUNS-C  WKTS  AVE-BL   ECON  \
0           0      0.00  ...    0.00       0     307    15   20.47   8.90   
1         657     71.41  ...    0.00       0      29     0    0.00  14.50   
2        1269     80.62  ...  121.01       5    1059    29   36.52   8.81   
3         241     84.56  ...   76.32       0    1125    49   22.96   6.23   
4          79     45.93  ...  120.71      28       0     0    0.00   0.00   

   SR-BL  AUCTION YEAR  BASE PRICE  SOLD PRICE  
0  13.93          2009